# Convert the XML tracklet files to kitti format txt labels files

These txt label files are added to the regular data files (not the stationary folders)

## Imports

In [44]:
import os
from pathlib import Path
import xml.etree.ElementTree as ET

## Constants and strings

In [46]:
DATA_DIR_ROOT = '../data'

## Go through each data folder

In [48]:
os.listdir(DATA_DIR_ROOT)

['2011_09_26_drive_0017_sync',
 '2011_09_26_drive_0018_sync',
 '2011_09_26_drive_0051_sync',
 '2011_09_26_drive_0060_sync',
 '2011_09_26_drive_0001_sync',
 '2011_09_26_drive_0002_sync',
 'calib_cam_to_cam.txt',
 'calib_imu_to_velo.txt',
 'calib_velo_to_cam.txt',
 '.ipynb_checkpoints',
 '2011_09_26_drive_0005_sync',
 '2011_09_26_drive_0009_sync',
 '2011_09_26_drive_0011_sync',
 '2011_09_26_drive_0013_sync',
 '2011_09_26_drive_0014_sync',
 '2011_09_26_drive_0015_sync',
 '2011_09_26_drive_0019_sync',
 '2011_09_26_drive_0020_sync',
 '2011_09_26_drive_0022_sync',
 '2011_09_26_drive_0059_sync',
 '2011_09_26_drive_0061_sync',
 '2011_09_26_drive_0064_sync',
 '2011_09_26_drive_0070_sync',
 '2011_09_26_drive_0079_sync',
 '2011_09_26_drive_0084_sync',
 '2011_09_26_drive_0086_sync',
 '2011_09_26_drive_0087_sync',
 '2011_09_26_drive_0091_sync',
 '2011_09_26_drive_0093_sync',
 '2011_09_26',
 '2011_09_26_drive_0023_sync',
 '2011_09_26_drive_0027_sync',
 '2011_09_26_drive_0028_sync',
 '2011_09_26_driv

In [49]:
# Make dictionary with an entry for each frame of sequence
def get_initialized_frame_dict(frame_data_path):
    # Make a dictionary
    label_dict = dict()
    
    # For each file in the folder
    for child in frame_data_path.iterdir():
        # Get the file name without the extension
        # print(child.name)
        frame_index = child.name[:-4]
        
        # If the file name is an integer
        if frame_index.isdigit():
            # Then use it to add an entry to the dictionary (an empty list)
            label_dict[frame_index] = []

    return label_dict

In [50]:
def get_string_from_roi_dict(roi_dict):
    line = ''
    for key in roi_dict:
        if type(roi_dict[key]) == str:
            line = line + ' ' + roi_dict[key]
        else:
            line = line + ' ' + str(round(roi_dict[key], 2))
    return line

In [51]:
# Function to make labels

'''
object_type, truncation, occlusion, observation_angle, ...
bbox(1), bbox(2), bbox(3), bbox(4), dimensions(1), dimensions(2), dimensions(3), ...
location(1), location(2), location(3), rotation_y)

type = <objectType>
truncated = <truncation>
occluded = <occlusion>
alpha = N/A → defaults to -10 (?)
bbox = N/A → defaults to 0.0 0.0 0.0 0.0
dimensions = <h> <w> <l>
location = <tx> <ty> <tz>
rotation_y = <ry>
score = N/A
'''
def populate_frame_label_dict(xml_path, label_dict):
    print()
    # Make entry dictionary with 0 values
    roi_dict = {
        'type': '',
        'truncated': 0.0,
        'occluded': 0,
        'alpha': 0.0,
        'bbox_0': 0.0,
        'bbox_1': 0.0,
        'bbox_2': 0.0,
        'bbox_3': 0.0,
        'dimensions_h': 0.0,
        'dimensions_w': 0.0,
        'dimensions_l': 0.0,
        'location_tx': 0.0,
        'location_ty': 0.0,
        'location_tz': 0.0,
        'rotation_y': 0.0
    }
    
    print(xml_path)
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # get tracklets
    tracklets = root[0]
    # print(tracklets)
    
    stem_entry = ''
    
    for car_item in tracklets.findall('item'):
        # Each item is a car,
        # print(car_item)
        
        # Get type
        type = car_item.find('objectType').text
        roi_dict['type'] = type
        
        # Get height and add to stem entry
        height = car_item.find('h').text
        roi_dict['dimensions_h'] = float(height)
        
        # Get width
        width = car_item.find('w').text
        # roi_dict['dimensions_w'] = float(width)
        roi_dict['dimensions_l'] = float(width)
        
        # Get length
        length = car_item.find('l').text
        # roi_dict['dimensions_l'] = float(length)
        roi_dict['dimensions_w'] = float(length)
        
        # Get current frame
        current_frame = int(car_item.find('first_frame').text)
        # print('current frame: ' + str(current_frame))
        
        poses = car_item.find('poses')
        # print(poses)
        
        frames = poses.findall('item')
        # print('_______________________________________________')
        # print(frames)
        # print()
        
        # For each frame of this object
        for frame in frames:
            # print(frame)
            
            # Get this location,
            # roi_dict['location_tx'] = float(frame.find('tx').text)
            # roi_dict['location_ty'] = float(frame.find('ty').text)
            # roi_dict['location_tz'] = float(frame.find('tz').text)
            roi_dict['location_tx'] = -float(frame.find('ty').text)
            roi_dict['location_ty'] = float(frame.find('tz').text)
            roi_dict['location_tz'] = float(frame.find('tx').text)
            
            # Get the rotation
            roi_dict['rotation_y'] = -float(frame.find('rz').text)
            
            # print('roi_dict: ', end='')
            # print(roi_dict)
            
            # Get string from roi
            roi_string = get_string_from_roi_dict(roi_dict)
            # Get frame with padded zeros
            frame_key = str(current_frame).zfill(10)
            # print(frame_key)
            # Add to label dict at correct frame index
            label_dict[frame_key].append(roi_string)

            # Increment frame
            current_frame += 1

            # Skip skipped frames
            while str(current_frame).zfill(10) not in label_dict.keys() and current_frame < 5000:
                current_frame += 1
    
    

In [52]:
def save_labels_to_dir(frame_dict, label_dir_path):
    # For each frame
    for key in frame_dict:
        # Start a new file with that name
        new_frame_file_name = str(label_dir_path) + '\\' + key + '.txt'
        # print(new_frame_file_name)

        # Start new file for frame
        with open(new_frame_file_name, 'w') as file:
            # Append each roi for the frame
            for index, line in enumerate(frame_dict[key]):
                if index != 0:
                    file.write('\n')
                file.write(line)

In [53]:
# This function creates a dictionary for each frame in the sequence,
# then populates the dictionary's list (value) with the frame's bounding boxes
# Then, for each frame (key) in the dictionary, it creates and saves a text file of the frame's
# bounding boxes

def get_sequence_labels(sequence_dir, label_dir):  
    print(sequence_dir)
    # Create and initialized dictionary
    velodyne_path = Path(sequence_dir, 'velodyne_points', 'data')
    frame_dict = get_initialized_frame_dict(velodyne_path)
    
    for child in sequence_dir.iterdir():
        # Populate dictionary with labels from xml file
        if str(child)[-4:] == '.xml':
            populate_frame_label_dict(child, frame_dict)

    # TODO: Save labels to directory
    save_labels_to_dir(frame_dict, label_dir)
    
    print()
    return frame_dict

In [54]:
p = Path(DATA_DIR_ROOT)

# For each sequence (folder) in the data
for child in p.iterdir(): 
    if child.is_dir() and 'drive' in str(child):
        
        # Add a folder for labels
        label_path = Path(child, 'labels')
        label_path.mkdir(exist_ok=True)
        
        # Save the labels for the sequence in the labels folder
        sequence_dir = Path(child)
        get_sequence_labels(sequence_dir, label_path)
        

..\data\2011_09_26_drive_0017_sync

..\data\2011_09_26_drive_0017_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0018_sync

..\data\2011_09_26_drive_0018_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0051_sync

..\data\2011_09_26_drive_0051_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0060_sync

..\data\2011_09_26_drive_0060_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0001_sync

..\data\2011_09_26_drive_0001_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0002_sync

..\data\2011_09_26_drive_0002_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0005_sync

..\data\2011_09_26_drive_0005_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0009_sync

..\data\2011_09_26_drive_0009_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0011_sync

..\data\2011_09_26_drive_0011_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0013_sync

..\data\2011_09_26_drive_0013_sync\tracklet_labels.xml

..\data\2011_09_26_drive_0014_sync

..\data\2011_09_26_drive_0014_sync\tracklet_

Do for each folder in the data folder